In [36]:
import pandas as pd
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie
from pythainlp.tag import pos_tag

from nltk.corpus import words
from stop_words import get_stop_words

In [37]:
# import data from csv
file_name = 'category1'

FILE = './Data/{}.csv'.format(file_name)
df = pd.read_csv(FILE)

In [38]:
df.head()

,Unnamed: 0,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,...,category_33_234,category_33_235,category_33_236,category_33_237,category_33_238,category_33_239,category_33_240,category_33_241,category_33_242,category_33_243
0,0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New
1,1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต
2,2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),...,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New
3,3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),...,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New
4,4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,...,GloryStar BASEUS PUBG ไฟฟรีเกมที่วางโทรศัพท์แผ...,GloryStar นาฬิกาติดตามตำแหน่งอัจฉริยะเด็กนาฬิก...,GloryStar Joy-CON ตัวควบคุมเกมแป้นเกมส์จอยเกมส...,GloryStar คอนโซลเกมคลาสสิก 8 BitsProfessional ...,GloryStar สีน้ำเงินแปรงเกมควบคุมเครื่องรับสัญญ...,GloryStar Flysky รีโมทคอนโทรล FS-IT4S + IA4B 4...,Top-Sky PlayStation 2 PS2 Memory Card - 32MB,Top-Sky PlayStation 2 PS2 Memory Card - 256MB,Top-Sky PlayStation 2 PS2 Memory Card - 128MB,Top - Sky 3 - in - 1 PS4/PS4 Slim/PS4 เกมโปรที...


In [39]:
# Drop a column that not use
data = df.drop(columns=['Unnamed: 0'])

# Drop column that all equal to NaN
data = data.dropna(axis=1, how='all')

# Fill NaN with "-"
data = data.fillna('-')

In [49]:
data.head(10)

,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,category_0_9,...,category_33_234,category_33_235,category_33_236,category_33_237,category_33_238,category_33_239,category_33_240,category_33_241,category_33_242,category_33_243
0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New
1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต
2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),Smartphone โทรศัพท์มือถือ Samsung Galaxy A10S ...,...,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New
3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),แบรนด์ ซัมซุง,...,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New
4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,SKU 571250946_TH-1048780375,...,GloryStar BASEUS PUBG ไฟฟรีเกมที่วางโทรศัพท์แผ...,GloryStar นาฬิกาติดตามตำแหน่งอัจฉริยะเด็กนาฬิก...,GloryStar Joy-CON ตัวควบคุมเกมแป้นเกมส์จอยเกมส...,GloryStar คอนโซลเกมคลาสสิก 8 BitsProfessional ...,GloryStar สีน้ำเงินแปรงเกมควบคุมเครื่องรับสัญญ...,GloryStar Flysky รีโมทคอนโทรล FS-IT4S + IA4B 4...,Top-Sky PlayStation 2 PS2 Memory Card - 32MB,Top-Sky PlayStation 2 PS2 Memory Card - 256MB,Top-Sky PlayStation 2 PS2 Memory Card - 128MB,Top - Sky 3 - in - 1 PS4/PS4 Slim/PS4 เกมโปรที...
5,ใช้ Exynos 9611 สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีส...,"4G FDD LTE B1(2100), B3(1800), B5(850), B7(260...",ระบบเซ็นเซอร์ (Sensor)- ระบบตรวจสอบลายนิ้วมือ ...,ความจุ (Storage) 256 GB,"มีสีให้เลือก (Colors) : Black, Pink, Gold, Blu...",กล้องดิจิตอล (Main Camera) 24MP + (Ultra Wide ...,กว้าง 6.4 นิ้ว (แนวทะแยง) - ความละเอียด 720 x ...,SKU 599364278_TH-1176694583,"หน้าจอกว้าง 6.7"" แบบ Infinity-U display",มือถือ Smartphone,...,แหล่งจ่ายไฟมือถือ 2000 mAh รองรับการชาร์จมือถื...,1.3 นิ้ว HD IPS จอแสดงผลสีจริง,รุ่นนี้สำหรับคอนโซลเปลี่ยนค่ะ ตัวควบคุมมีฟังก์...,"คอนโซลเกมคลาสสิก: 8 - bit เกมคอนโซล, เกมคลาสสิ...",ปรับความกว้าง 50-85 มม. ช่วง ..,2.4055 GHz ถึง 2.475 GHz กว้างแถบความถี่และ 14...,Can be used to save game data for any PS2 game...,Can be used to save game data for any PS2 game...,Can be used to save game data for any PS2 game...,"ที่ขาตั้งรองรับสำหรับ PS4 โฮสต์, และพอร์ตระบาย..."
6,หน้าจอแสดงผล Super AMOLED Infinity-U ขนาด 6.4 ...,"4G TDD LTE B38(2600), B40(2300), B41(2500)","มีสีให้เลือก (Colors) : Black, Red, Green, B

In [51]:
# Filter
data_01 = data.filter(regex ='category_[01]_[0-9]+')
data_01.head(20)

,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,category_0_9,...,category_1_234,category_1_235,category_1_236,category_1_237,category_1_238,category_1_239,category_1_240,category_1_241,category_1_242,category_1_243
0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,CHUWI Hi10 Air 2-in-1 Tablet+Laptop PC 10.1 นิ...,โทรศัพท์มือถือและแท็บเล็ต
1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แท็บเล็ต,แท็บเล็ต,แท็บเล็ต,แท็บเล็ต,แท็บเล็ต,แท็บเล็ต,แท็บเล็ต,แท็บเล็ต,"Asus Tablet ZENFONE Go 6.9 inch\r\n฿ 4,990.00",แท็บเล็ต
2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),Smartphone โทรศัพท์มือถือ Samsung Galaxy A10S ...,...,LENOVO TAB4 10 Plus (X704L) Tablet 10.1'' (4G)...,Lenovo แท็บเล็ต Tablet 7'' (WIFI) LENOVO TAB2 ...,"Tablet 7"" (3G CALL) LENOVO TAB4 (7104I) Black",LENOVO TAB E7 Ram1GB/Rom16GB 7.0 3G โทรได้,Tablet 10.1'' (4G) LENOVO TAB4 10 Plus (X704L)...,"Lenovo TAB4 Tablet 7"" TB-7104l (3G+ CALL) แถม ...",Lenovo TAB4 7104I (ZA410083TH) Android Tablet ...,"แท็บเล็ต Tablet 8'' (4G,CALL) LENOVO TAB3 (850...",I Pad pro 10.5 สวยมากๆๆจอใหญ่ดูหนังเล่นเกมส์ชั...,"LENOVO TAB4 7HD TB-7504X 7"" QC1.3 RAM2GB ROM16..."
3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),แบรนด์ ซัมซุง,...,CPU : Octa-core : 2.0GHz,หน้าจอ 7 นิ้ว ความละเอียด 1024 x 600 พิกเซล,CPU : Quad-Core : 1.3GHz,LENOVO TAB E7,Cpu : OctaCore 2.0GHz,แบรนด์ เลโนโว่,"3G with Voice (โทรได้) ,มีเคส+ฟิล์มในกล่อง",หน้าจอ 8 นิ้ว ความละเอียด 1280x800 พิกเซล,โน้ตบุ๊คและแท็บเล็ต Lenovo 2 in 1 Ideapad D330...,แบรนด์ เลโนโว่
4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,SKU 571250946_TH-1048780375,...,Ram: 4GB,CPU Quad Core 1.3 GHz,Ram : 1GB,Lenovo Tab E7หน้าจอ : LCD TN panel ขนาด 7.0 นิ...,Ram : 4GB,SKU 332512941_TH-641144040,แบรนด์ เลโนโว่,CPU Octa Core 1.0GHz,Original Huawei MediaPad M5 /M5 Pro 10.8 inch ...,SKU 404360545_TH-784536408
5,ใช้ Exynos 9611 สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีส...,"4G FDD LTE B1(2100), B3(1800), B5(850), B7(260...",ระบบเซ็นเซอร์ (Sensor)- ระบบตรวจสอบลายนิ้วมือ ...,ความจุ (Storage) 256 GB,"มีสีให้เลือก (Colors) : Black, Pink, Gold, Blu...",กล้องดิจิตอล (Main Camera) 24MP + (Ultra Wide ...,กว้าง 6.4 นิ้ว (แนวทะแยง) - ความละเอียด 720 x ...,SKU 599364278_TH-1176694583,"หน้าจอกว้าง 6.7"" แบบ Infinity-U display",มือถือ Smartphone,...,Rom: 64GB,RAM 1 GB,Rom : 16GB,CPU : MediaTek MT8167 chipset (quad-core 1.3GHz),Rom : 64GB,ประเภทของการรับประกัน มีการรับประกัน,SKU 487584299_TH-875784181,RAM 2GB,Alldocube Filp Case เคสฝาพับ สำหรับ Alldocube ...,ประเภทของการรับประกัน มีการรับประกัน
6,หน้าจอแสดงผล Super AMOLE

# Clean text

In [52]:
file_name2 = './Data/attributes_category1.txt'
with open(file_name2, 'r', encoding='utf-8') as f:
    attr_fix = f.read().splitlines()

In [53]:
# clean the text and prepre some attribute
def clean_text_category1(msg):
    msg = str(msg).lower()
    msg = re.sub(r"smartphone[s]?","สมาร์ทโฟน",str(msg))
    
    if str(msg)=='โทรศัพท์มือถือและแท็บเล็ต':
        msg = re.sub(r"(โทรศัพท์มือถือและแท็บเล็ต)",r"ประเภท \1",str(msg))
    elif (str(msg)=='สมาร์ทโฟน') | (str(msg)=='แท็บเล็ต'):
        msg = re.sub(r"(สมาร์ทโฟน|แท็บเล็ต)",r"ประเภทย่อย \1",str(msg))
    elif re.match("[0-9]\*[0-9]",str(msg)):
        msg = re.sub(r"\*","x",str(msg))
    
    msg = re.sub(r"\&","และ",str(msg))
    msg = re.sub(r"sku","sku ",str(msg))
    msg = re.sub(r"\%","เปอร์เซนต์ ",str(msg))
    msg = re.sub(r"brand","แบรนด์ ",str(msg))
    msg = re.sub(r"model","โมเดล ",str(msg))
    msg = re.sub(r"[\W]จอแสดงผล\s","หน้าจอแสดงผล ",str(msg))
    msg = re.sub(r"\sos\:?\s","ระบบปฏิบัติการ ",str(msg))
    msg = re.sub(r"ความจุ\s*\(storage\)","ความจุที่เก็บข้อมูล ",str(msg))
    msg = re.sub(r"[\d+\s+]mp|เมกะพิกเซล\s?","ล้านพิกเซล ",str(msg))
    msg = re.sub(r"\s+แรม\s?"," ram ",str(msg))
    msg = re.sub(r"\s+รอม\s?"," rom ",str(msg))
    msg = re.sub(r"ฮาร์ดดิสก์\s?|hard\s?disk\s?"," hdd ",str(msg))
    msg = re.sub(r"display","จอแสดงผล ",str(msg))
    msg = re.sub(r"weight?","น้ำหนัก ",str(msg))
    msg = re.sub(r"[b]attery\s?[c]apacity","ความจุแบตเตอรี่ ",str(msg))
    msg = re.sub(r"แบต\s?^เตอรี่","แบตเตอรี่ ",str(msg))
    msg = re.sub(r"\฿,"," บาท ",str(msg))
    msg = re.sub(r"[\d\s]*มิลลิเมตร|\smm\.?|millimeter\.?"," มม. ",str(msg))
    msg = re.sub(r"[\d\s]*เซนติเมตร|\scm\.?|centimeter\.?"," ซม. ",str(msg))
    msg = re.sub(r"[\,]","",str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\s]"," ", str(msg))
    msg = re.sub(r"\s+"," ",str(msg))
    if any(ext in msg for ext in attr_fix):
        return msg
    else:
        return

In [58]:
# clean name
def clean_name(msg):
    msg = str(msg).lower()
    
    msg = re.sub(r"\&","และ",str(msg))
    msg = re.sub(r"sku","sku ",str(msg))
    msg = re.sub(r"brand","แบรนด์ ",str(msg))
    msg = re.sub(r"model","โมเดล ",str(msg))
    msg = re.sub(r"[\W]จอแสดงผล\s","หน้าจอแสดงผล ",str(msg))
    msg = re.sub(r"\sos\:?\s","ระบบปฏิบัติการ ",str(msg))
    msg = re.sub(r"ความจุ\s*\(storage\)","ความจุที่เก็บข้อมูล ",str(msg))
    msg = re.sub(r"[\d+\s+]mp|เมกะพิกเซล\s?","ล้านพิกเซล ",str(msg))
    msg = re.sub(r"\s+แรม\s?"," ram ",str(msg))
    msg = re.sub(r"\s+รอม\s?"," rom ",str(msg))
    msg = re.sub(r"ฮาร์ดดิสก์\s?|hard\s?disk\s?"," hdd ",str(msg))
    msg = re.sub(r"display","จอแสดงผล ",str(msg))
    msg = re.sub(r"weight?","น้ำหนัก ",str(msg))
    msg = re.sub(r"[b]attery\s?[c]apacity","ความจุแบตเตอรี่ ",str(msg))
    msg = re.sub(r"แบต\s?^เตอรี่","แบตเตอรี่ ",str(msg))
    msg = re.sub(r"[\d\s]*มิลลิเมตร|\smm\.?|millimeter\.?"," มม. ",str(msg))
    msg = re.sub(r"[\d\s]*เซนติเมตร|\scm\.?|centimeter\.?"," ซม. ",str(msg))
    msg = re.sub(r"[\,]","",str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\s]"," ", str(msg))
    msg = re.sub(r"\s+"," ",str(msg))
    
    return msg
    

In [54]:
# function for defining attribute of product
def define_attr(txt):
    p = re.compile('(ประเภท\s|ประเภทย่อย|แบรนด์|รุ่น|กล้องหน้า|กล้องหลัง|โมเดล|รูรับแสงขนาด|\
                ขนาดหน้าจอ|หน่วยประมวลผล|แบตเตอรี่|น้ำหนัก|sku|ระยะเวลาการรับประกัน|\
                ระบบปฏิบัติการ|ความจุแบตเตอรี่|หน้าจอแสดงผล|ram|rom|ความจุ|\
                มาตรฐานการกันน้ำและกันฝุ่น|รองรับ|gpu|cpu|ความละเอียด|หน้าจอ|ขนาด)', re.VERBOSE)
    #match = re.match(r"", str(txt))
    txt = re.sub(p,r"|\1|",str(txt))
    
    return txt

In [59]:
# Cleaning text with clean_text function and keep in dict
data_clean = {}
name_item = {}
for key,value in data_01.iteritems():
    tmp = []
    for i,txt in enumerate(value):
        if not txt == '-':
            if i==2:
                name_item[key] = clean_name(txt)
            tmp.append(clean_text_category1(txt))
    
    tmp = [i for i in tmp if i] 
    data_clean[key] = tmp 


In [60]:
# Show some of data in dict
name_item

{'category_0_0': 'samsung galaxy m30s 4 64gb โทรศัพท์มือถือ ',
 'category_0_1': 'โทรศัพท์มือถือ samsung galaxy a50s ram4 64gb รับประกันศูนย์ by jaymart',
 'category_0_2': 'samsung galaxy a10s ram2gb rom32gb ',
 'category_0_3': 'samsung galaxy note 10 12 256 gb โทรศัพท์มือถือ ',
 'category_0_4': 'samsung galaxy a7 2017 ',
 'category_0_5': 'samsung galaxy a9 2018 6 gb ram 128 gb rom 6.3 นิ้ว 5 กล้อง สูงสุด 24ล้านพิกเซล tetra cell f1.7 โทรศัพท์มือถือ ',
 'category_0_6': 'samsung a30s ram 4gb rom 64 gb มือ 1 ประกันศูนย์ 1 ปี',
 'category_0_7': ' มือ1 ศูนย์ไทย samsung galaxy note 10 plus note 10 12 256gb ประกัน1ปี',
 'category_0_8': 'samsung galaxy a70 8 128gb โทรศัพท์มือถือ ',
 'category_0_9': 'smartphone โทรศัพท์มือถือ samsung galaxy a10s sm a107m ขนาดหน้าจอ 6.2 นิ้ว ram 2 rom 32 gb สีน้ำเงิน ',
 'category_0_10': 'samsung galaxy a5 2017 ',
 'category_0_11': 'samsung galaxy a5 2017 gold',
 'category_0_12': 'samsung galaxy s10e 6 128gb โทรศัพท์มือถือ ',
 'category_0_13': 'smartphone โทรศัพท

In [29]:
# Mark attribute with '|'
attr_mark = {}
for key,value in data_clean.items():
    tmp = [define_attr(txt) for txt in value] 
    tmp = [re.sub(r"\|\|","",str(t)) for t in tmp]
    tmp = [i for i in tmp if re.search(r"\|",i)] 
    attr_mark[key] = tmp 

In [30]:
attr_mark

{'category_34_0': ['|ประเภท| โทรศัพท์มือถือและแท็บเล็ต',
  'รูปลักษณ์สวยงามหลากห|ลายสี|ที่ให้เลือกจาก',
  '|แบรนด์| oem',
  '|ประเภทของการรับประกัน| ไม่มีการประกัน',
  '|รุ่น| lqlkwkk0017'],
 'category_34_1': ['|ประเภท| โทรศัพท์มือถือและแท็บเล็ต',
  'ออกแบบแฟขั่นช่วยให้โทรศัพท์ของคุณส่วนตัวและรู้สึก goodshock ดูดซับและ|วัสดุ|กันรอยขีดข่วน.',
  '|แบรนด์| no |แบรนด์| ',
  '|ประเภทของการรับประกัน| ไม่มีการประกัน',
  '|ประเภท|ของเคส ฝาหลัง',
  '|แบรนด์|ที่ใช้ร่วมได้ แอปเปิ้ล'],
 'category_34_2': ['|ประเภท| โทรศัพท์มือถือและแท็บเล็ต',
  'เคสกำมะหยี่ โลโก้ apple |รุ่น| iphone6 6s iphone7 8 6plus 6splus 7plus 8plus iphonex xs xr xsmax iphone11 11pro 11pro max เคสซิริโคนกำมะหยี่',
  'sunny case ยินดีให้บริการลูกค้าทุกท่านค่ะ ร้านของเราเป็นร้านขายส่ง เคสโทรศัพท์มือถือทุก|รุ่น| มีห|ลาย|แบบ ห|ลายรุ่น|ให้ลูกค้าเลือกซื้อ เลือกชม ลูกค้าสามารถสอบถามรายละเอียดเพิ่มเติมได้นะคะ',
  'ทางร้านรับชำระเงินป|ลาย|ทางค่ะ',
  '|แบรนด์| no |แบรนด์| ',
  '|ประเภทของการรับประกัน| มีการรับประกันจากผู้ขาย',
  '|ระยะเ

# Capture attribute manual

In [31]:
# Split attribute
def cap_attr(text):
    text = text.split("|")
    
    return text

In [32]:
# capture attribute by split the marker
attr = {}
for i,v in attr_mark.items():
    tmp_list = []
    for txt in v:
        match = cap_attr(txt)
        match = [m for m in match if not m =='' and not m == ' ']
        tmp_list.append(match)
    attr[i] = tmp_list

In [33]:
attr

{'category_34_0': [['ประเภท', ' โทรศัพท์มือถือและแท็บเล็ต'],
  ['รูปลักษณ์สวยงามหลากห', 'ลายสี', 'ที่ให้เลือกจาก'],
  ['แบรนด์', ' oem'],
  ['ประเภทของการรับประกัน', ' ไม่มีการประกัน'],
  ['รุ่น', ' lqlkwkk0017']],
 'category_34_1': [['ประเภท', ' โทรศัพท์มือถือและแท็บเล็ต'],
  ['ออกแบบแฟขั่นช่วยให้โทรศัพท์ของคุณส่วนตัวและรู้สึก goodshock ดูดซับและ',
   'วัสดุ',
   'กันรอยขีดข่วน.'],
  ['แบรนด์', ' no ', 'แบรนด์'],
  ['ประเภทของการรับประกัน', ' ไม่มีการประกัน'],
  ['ประเภท', 'ของเคส ฝาหลัง'],
  ['แบรนด์', 'ที่ใช้ร่วมได้ แอปเปิ้ล']],
 'category_34_2': [['ประเภท', ' โทรศัพท์มือถือและแท็บเล็ต'],
  ['เคสกำมะหยี่ โลโก้ apple ',
   'รุ่น',
   ' iphone6 6s iphone7 8 6plus 6splus 7plus 8plus iphonex xs xr xsmax iphone11 11pro 11pro max เคสซิริโคนกำมะหยี่'],
  ['sunny case ยินดีให้บริการลูกค้าทุกท่านค่ะ ร้านของเราเป็นร้านขายส่ง เคสโทรศัพท์มือถือทุก',
   'รุ่น',
   ' มีห',
   'ลาย',
   'แบบ ห',
   'ลายรุ่น',
   'ให้ลูกค้าเลือกซื้อ เลือกชม ลูกค้าสามารถสอบถามรายละเอียดเพิ่มเติมได้นะคะ'],
  ['ทางร้า

In [34]:
# Construct triples of product
triples = []
for i,v in attr.items():
    for l in v:
        idx=0
        for idx in range(len(l)-1):
            tmp_list = [i]
            if l[idx] in attr_fix:
                tmp_list.extend([l[idx],l[idx+1]])
                triples.append(tmp_list)
            idx += 1     

In [35]:
triples

[['category_34_0', 'ประเภท', ' โทรศัพท์มือถือและแท็บเล็ต'],
 ['category_34_0', 'แบรนด์', ' oem'],
 ['category_34_0', 'ประเภทของการรับประกัน', ' ไม่มีการประกัน'],
 ['category_34_0', 'รุ่น', ' lqlkwkk0017'],
 ['category_34_1', 'ประเภท', ' โทรศัพท์มือถือและแท็บเล็ต'],
 ['category_34_1', 'วัสดุ', 'กันรอยขีดข่วน.'],
 ['category_34_1', 'แบรนด์', ' no '],
 ['category_34_1', 'ประเภทของการรับประกัน', ' ไม่มีการประกัน'],
 ['category_34_1', 'ประเภท', 'ของเคส ฝาหลัง'],
 ['category_34_1', 'แบรนด์', 'ที่ใช้ร่วมได้ แอปเปิ้ล'],
 ['category_34_2', 'ประเภท', ' โทรศัพท์มือถือและแท็บเล็ต'],
 ['category_34_2',
  'รุ่น',
  ' iphone6 6s iphone7 8 6plus 6splus 7plus 8plus iphonex xs xr xsmax iphone11 11pro 11pro max เคสซิริโคนกำมะหยี่'],
 ['category_34_2', 'รุ่น', ' มีห'],
 ['category_34_2', 'ลาย', 'แบบ ห'],
 ['category_34_2', 'ลาย', 'ทางค่ะ'],
 ['category_34_2', 'แบรนด์', ' no '],
 ['category_34_2', 'ประเภทของการรับประกัน', ' มีการรับประกันจากผู้ขาย'],
 ['category_34_2', 'ระยะเวลาการรับประกัน', ' 3 วัน'],
 [

In [227]:
# Write the triples to text file
with open('./Data/triples_01.txt', 'w', encoding='utf-8') as f:
    for _list in triples:
        for _string in _list:
            #f.seek(0)
            f.write(str(_string) + '\t')
        f.write('\n')

In [62]:
with open('./Data/Product_names.txt', 'w', encoding='utf-8') as fn:
    print(name_item, file=fn)

# Proprocessing text

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# path of custom dictionary for word tokenize
PATH_TO_CUSTOM_DICT = './Data/custom_dict.txt'

from pythainlp.tokenize import sent_tokenize

# split text to words 
def split_word(text):
    
    # Split text to words
    _tokenizer = Tokenizer(custom_dict=PATH_TO_CUSTOM_DICT, engine='newmm')
    words = sent_tokenize(text)

    # Remove stop words in TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]
    
    # Delete whitespace
    words = [i for i in words if not ' ' in i]
                

    return words

## Tokenize text

token = {}
for key,value in data_clean.items():
    tmp = [split_word(txt) for txt in value]
    token[key] = tmp

token_list = {}
for i,v in token.items():
    token_list[i] = [item for sublist in v for item in sublist]

attr_list = {}
res = []
for i,v in attr.items():
    tmp_list = []
    for t in v:
        if type(t) != list:
            t_split = t.split(':')
            if len(t_split)<3:
                tmp_list.append(t_split)
        else:
            for ele in t:
                if (ele[0]!='' and ele[1]!=''):
                    res = [list(ele)]
            
            for l in res:
                tmp_list.append(l)
    attr_list[i] = tmp_list

attr_list

dict_attr = {}
dict_value = {}
for i,v in attr_list.items():
    tmp_attr = []
    tmp_value = []
    for e in v:
        tmp_attr.append(e[0])
        tmp_value.append(e[1])
    dict_attr[i] = tmp_attr
    dict_value[i] = tmp_value

dict_value

series = { key: pd.Series(val) for key, val in token.items() }

token2 = {}
for i,v in series.items():
    token2[i] = v.apply(pd.Series).stack()

pd.DataFrame.from_dict(token2, orient='index').transpose()

## Category 1

token2

# Save data to csv file

# Transform dict to dataframe and save to csv file
pd.DataFrame.from_dict(data_clean, orient='index').transpose().reset_index().to_csv("./Data/{}_clean.csv".format(file_name))

pd.DataFrame.from_dict(data_clean, orient='index').transpose().reset_index().to_excel('./Data/category1_clean.xlsx',engine='xlsxwriter')

pd.DataFrame.from_dict(dict_value, orient='index').transpose().to_csv("./Data/{}_value.csv".format(file_name))